In [2]:

import numpy as np
from dwave.system import LeapHybridBQMSampler, LeapHybridCQMSampler
from neal import SimulatedAnnealingSampler
import pandas as pd


In [3]:
bqm_solver = LeapHybridBQMSampler(token= 'DEV-bffc84fbc5334f436b32767dfbf3ea1ea603e152') #my id, change to give like a real imput when doing the real thing

In [4]:
class Med:
    # Constructor method to initialize the car with a make, model, and year
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
    
    def give_ks(self):
        return (self.value, self.weight)
    
    

    

In [5]:

dataset = {'sleepaid': Med(100, 4),
            'retinol': Med(200, 5),
            'Aspirin': Med(250, 3),
            'Imuran': Med(1000, 5),
            "Liver albumin" : Med(924, 7),
            "Acitretin Capsules" : Med(200, 5),
            "Tretinon" : Med(230, 9)}


values = [dataset[d].value for d in dataset.keys()] 
weight = [dataset[d].weight for d in dataset.keys()] 

v= pd.DataFrame(values)
w= pd.DataFrame(weight)



In [12]:

#j in 0, 1, ..., N-1
#wj = weight, vj = value || j

# i in 0, 1, 2, ... M-1
# ci = capacity of the ies knapsack
#x_{i, j} = 1 if j is assigned to i
from dimod import BQM

def bqm_multiple_knapsack(num_med, num_trav, dataset=None, A = 2, B = 2, C= 2):
    if not dataset:
        weight = [i for i in range(num_med)]
        values = [3 * i for i in range(num_trav)]

        dataset = {'sleepaid': Med(100, 4),
            'retinol': Med(200, 5),
            'Aspirin': Med(250, 3),
            'Imuran': Med(1000, 5),
            "Liver albumin" : Med(924, 7),
            "Acitretin Capsules" : Med(200, 5),
            "Tretinon" : Med(230, 9)}


        #values = [dataset[d].value for d in dataset.keys()] 
        #weight = [dataset[d].weight for d in dataset.keys()] 
        
        capacity = [4 * j + 5 for j in range(num_trav)]
    
    else:
        #fill this
        pass


    #for formulate the QUBO instance

    #first Constraint: Any item j can be assigned to at most one knapsack. 
    # It is possible that an item is not assigned to any knapsack.
    
    bqm = BQM('BINARY')
    
    quadratic1 = {} 

    for j in range(num_med):
        for i in range(num_trav):
            for k in range(i):
                
                quadratic1[(f"x_{i}_{j}", f"x_{k}_{j}")] = 2 * A


    #second constraint: Ensure that no knapsack’s capacity is exceeded
    linear2 = {}
    quadratic2 = {}

    offset = 0
    for i in range(num_trav):
        ci = capacity[i]
        for j, b in zip(range(num_med), range( int(np.floor( np.log2(ci) ) ))):
            wj = weight[j]

            linear2[f'x_{i}_{j}'] = (wj ** 2 - 2 * ci * wj) * B
            linear2[f'y_{i}_{b}'] = (2 ** (2 * b) - ci * 2 ** (b + 1)) * B
            if i != j:
                quadratic2[(f'x_{i}', f'x_{j}')] = (wj * ( 2 ** (b + 1))) * B
    
        offset += (ci ** 2) * B


    #third constraint: The objective term is formulated such that our 
    # original maximization objective function is converted to a 
    # minimization problem.

    linear3 = {}
    for i in range(num_trav):

        for j in range(num_med):
            linear3[f'x_{i}_{j}'] = (-1 * values[i]) * C
    
    bqm.add_linear_from(linear2)
    bqm.add_linear_from(linear3)
    bqm.add_quadratic_from(quadratic1)
    bqm.add_quadratic_from(quadratic2)
    bqm.add_offset(offset)
    
    return bqm

    
    





In [13]:
N = 10 #number of medicines
M = 3 #number of travellers

bqm_form = bqm_multiple_knapsack(num_med= N, num_trav=M)

sampleset = bqm_solver.sample(bqm_form)

print(sampleset.first.sample)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14784\290279768.py:81: DeprecationWarning: BinaryQuadraticModel.add_offset(bias) is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Please use bqm.offset += bias.
  bqm.add_offset(offset)


{'x_0': 1, 'x_0_0': 0, 'x_0_1': 1, 'x_0_2': 0, 'x_0_3': 0, 'x_0_4': 0, 'x_0_5': 0, 'x_0_6': 0, 'x_0_7': 0, 'x_0_8': 0, 'x_0_9': 0, 'x_1': 0, 'x_1_0': 1, 'x_1_1': 1, 'x_1_2': 1, 'x_1_3': 1, 'x_1_4': 1, 'x_1_5': 1, 'x_1_6': 1, 'x_1_7': 1, 'x_1_8': 1, 'x_1_9': 1, 'x_2': 1, 'x_2_0': 1, 'x_2_1': 1, 'x_2_2': 1, 'x_2_3': 1, 'x_2_4': 1, 'x_2_5': 1, 'x_2_6': 1, 'x_2_7': 1, 'x_2_8': 1, 'x_2_9': 1, 'y_0_0': 1, 'y_0_1': 1, 'y_1_0': 1, 'y_1_1': 1, 'y_1_2': 1, 'y_2_0': 1, 'y_2_1': 1, 'y_2_2': 1}


this is read as it follows:
0 traveler packs up medicine 1
1 traveler packs up med [0 to 9]
2 traveler packs up med [0 to 9]